In [ ]:
# 决策树的优点：

# 决策树算法中学习简单的决策规则建立决策树模型的过程非常容易理解，
# 决策树模型可以可视化，非常直观
# 应用范围广，可用于分类和回归，而且非常容易做多类别的分类
# 能够处理数值型和连续的样本特征

In [ ]:
# 决策树的缺点：

# 很容易在训练数据中生成复杂的树结构，造成过拟合（overfitting）。剪枝可以缓解过拟合的负作用，常用方法是限制树的高度、叶子节点中的最少样本数量。
# 学习一棵最优的决策树被认为是NP-Complete问题。实际中的决策树是基于启发式的贪心算法建立的，这种算法不能保证建立全局最优的决策树（Random Forest 引入随机能缓解这个问题）。

In [ ]:
# 基于ID3算法的决策分析

# ID3是由J.Ross Quinlan在1986年开发的一种基于决策树的分类算法。该算法是以信息论为基础，以信息熵核信息增益为衡量标准，从而实现对数据的归纳分类。根据信息增益运用自顶向下的贪心策略是ID3建立决策树的主要方法。运用ID3算法的主要优点是建立的决策树的规模比较小，查询速度比较快。这个算法建立在“奥卡姆剃刀”的基础上，即越是小型的决策树越优于大的决策树。但是，该算法在某些情况下生成的并不是最小的树型结构

In [ ]:
# 信息量

# 信息量在是作为信息“多少”的度量，这里的信息就是你理解的信息，比如一条新闻，考试答案等等。假设我们听到了两件事，分别如下：

# 事件A：巴西队进入了2018世界杯决赛圈。
# 事件B：中国队进入了2018世界杯决赛圈。
# 仅凭直觉来说，事件B的信息量比事件A的信息量要大。究其原因，是因为事件A发生的概率很大，事件B发生的概率很小。所以当越不可能的事件发生了，我们获取到的信息量就越大。越可能发生的事件发生了，我们获取到的信息量就越小。那么：

# 信息量和事件发生的概率相关，事件发生的概率越低，传递的信息量越大
# 信息量应当是非负的，必然发生的事件的信息量为零（必然事件是必然发生的，所以没有信息量。几乎不可能事件一旦发生，具有近乎无穷大的信息量。）
# 两个事件的信息量可以相加，并且两个独立事件的联合信息量应该是他们各自信息量的和

In [ ]:
# 信息熵

# 信息熵（Entropy）是接受信息量的平均值，用于确定信息的不确定程度，是随机变量的均值。信息熵越大，信息就越凌乱或传输的信息越多，熵本身的概念源于物理学中描述一个热力学系统的无序程度。信息熵的处理信息是一个让信息的熵减少的过程。

In [ ]:
# 条件熵

# 在决策树的切分里，事件 x_i 可以认为是在样本中出现某个标签/决策。于是 P(x_i)可以用所有样本中某个标签出现的频率来代替。但我们求熵是为了决定采用哪一个维度进行切分，因此有一个新的概念条件熵：

In [ ]:
# ID3算法流程

# 输入：数据集D，特征集A

# 输出：ID3决策树

# 对当前样本集合计算出所有属性信息的信息增益
# 先选择信息增益最大的属性作为测试属性，将测试属性相同的样本转化为同一个子样本
# 若子样本集的类别属性只含有单个属性，则分支为叶子节点，判断其属性值并标上相应的符号，然后返回调用处，否则对子样本递归调用本算法

In [ ]:
# ID3算法的Python实现

In [2]:
from math import log
import operator

# 计算香农熵
def calculate_entropy(data):
    label_counts = {}
    for feature_data in data:
        laber = feature_data[-1] # 最后一行是laber
        if laber not in label_counts.keys():
            label_counts[laber] = 0
        label_counts[laber] += 1
        
    count = len(data)
    entropy = 0.0
    
    for key in label_counts:
        prob = float(label_counts[key]) / count
        entropy -= prob * log(prob, 2)
    return entropy

In [3]:
# 计算某个feature的信息增益
# index：要计算信息增益的feature对应的再data的第几列
# data 的香农熵

def calculate_relative_entropy(data, index, entropy):
    feat_list = [number[index] for number in data]  # 得到某个特征下所有值（某列）
    uniqual_vals = set(feat_list)
    new_entropy = 0
    for value in uniqual_vals:
        sub_data = split_data(data, index, value)
        prob = len(sub_data) / float(len(data))  
        new_entropy += prob * calculate_entropy(sub_data)  # 对各子集香农熵求和
    relative_entropy = entropy - new_entropy  # 计算信息增益
    return relative_entropy

In [4]:
# 选择最大信息增益的feature
def choose_max_relative_entropy(data):
    num_feature = len(data[0]) - 1
    base_entroy = calculate_entropy(data) # 香农熵
    best_infor_gain = 0
    for i in range(num_feature):
        info_again = calculate_relative_entropy(data, i, base_entroy)
        #最大信息增益
        if (info_again > best_infor_gain):
            best_infor_gain = info_again
            best_feature = i
    return best_feature

In [9]:
def create_decision_tree(data, labels):
    class_list=[example[-1] for example in data]
    # 类别相同，停止划分
    if class_list.count(class_list[-1]) == len(class_list):
        return class_list[-1]
    # 判断是否遍历完所有的特征时返回个数最多的类别
    if len(data[0]) == 1:
        return most_class(class_list)
    # 按照信息增益最高选取分类特征属性
    best_feat = choose_max_relative_entropy(data)
    best_feat_lable = labels[best_feat] # 该特征的label
    decision_tree = {best_feat_lable: {}} # 构建树的字典
    del(labels[best_feat]) # 从labels的list中删除该label
    feat_values = [example[best_feat] for example in data]
    unique_values = set(feat_values)
    for value in unique_values:
        sub_lables=labels[:]
        # 构建数据的子集合，并进行递归
        decision_tree[best_feat_lable][value] = create_decision_tree(split_data(data, best_feat, value), sub_lables)
    return decision_tree
 
# 当遍历完所有的特征时返回个数最多的类别
def most_class(classList):
    class_count={}
    for vote in classList:
        if vote not in class_count.keys():class_count[vote]=0
        class_count[vote]+=1
    sorted_class_count=sorted(class_count.items,key=operator.itemgetter(1),reversed=True)
    return sorted_class_count[0][0]
    
# 工具函数输入三个变量（待划分的数据集，特征，分类值）返回不含划分特征的子集
def split_data(data, axis, value):
    ret_data=[]
    for feat_vec in data:
        if feat_vec[axis]==value :
            reduce_feat_vec=feat_vec[:axis]
            reduce_feat_vec.extend(feat_vec[axis+1:])
            ret_data.append(reduce_feat_vec)
    return ret_data

In [ ]:
# ID3算法的优点与缺点

# ID3算法的优点：

# 算法结构简单；
# 算法清晰易懂；
# 非常灵活方便；
# 不存在无解的危险；
# 可以利用全部训练例的统计性质进行决策，从而抵抗噪音。
# ID3算法简单，但是其缺点也不少：

# ID3算法采用信息增益来选择最优划分特征，然而人们发现，信息增益倾向与取值较多的特征，对于这种具有明显倾向性的属性，往往容易导致结果误差；
# ID3算法没有考虑连续值，对与连续值的特征无法进行划分；
# ID3算法无法处理有缺失值的数据；
# ID3算法没有考虑过拟合的问题，而在决策树中，过拟合是很容易发生的；
# ID3算法采用贪心算法，每次划分都是考虑局部最优化，而局部最优化并不是全局最优化，当然这一缺点也是决策树的缺点，获得最优决策树本身就是一个NP难题，所以只能采用局部最优；